In [ ]:
pip install transformers

     |████████████████████████████████| 3.3 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 65.6 MB/s 
     |████████████████████████████████| 3.3 MB 63.6 MB/s 
     |████████████████████████████████| 596 kB 74.9 MB/s 
     |████████████████████████████████| 61 kB 527 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json -O KorQuAD_v1.0_train.json
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json -O KorQuAD_v1.0_dev.json

--2021-12-10 05:50:32--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M   197MB/s    in 0.2s    

2021-12-10 05:50:32 (197 MB/s) - ‘KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2021-12-10 05:50:32--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[======

In [ ]:
import os
import json
import numpy as np
from tqdm import tqdm
from pathlib import Path
from transformers import BertTokenizerFast
import tensorflow as tf

In [ ]:
def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('KorQuAD_v1.0_train.json')
val_contexts, val_questions, val_answers = read_squad('KorQuAD_v1.0_dev.json')

In [ ]:
print('훈련 데이터의 본문 개수 :', len(train_contexts))
print('훈련 데이터의 질문 개수 :', len(train_questions))
print('훈련 데이터의 답변 개수 :', len(train_answers))
print('테스트 데이터의 본문 개수 :', len(val_contexts))
print('테스트 데이터의 질문 개수 :', len(val_questions))
print('테스트 데이터의 답변 개수 :', len(val_answers))

훈련 데이터의 본문 개수 : 60407
훈련 데이터의 질문 개수 : 60407
훈련 데이터의 답변 개수 : 60407
테스트 데이터의 본문 개수 : 5774
테스트 데이터의 질문 개수 : 5774
테스트 데이터의 답변 개수 : 5774


In [ ]:
print('첫번째 샘플의 본문')
print('-----------------')
print(train_contexts[0])

첫번째 샘플의 본문
-----------------
1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.


In [ ]:
print('첫번째 샘플의 질문')
print('-----------------')
print(train_questions[0])

첫번째 샘플의 질문
-----------------
바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?


In [ ]:
print('첫번째 샘플의 답변')
print('-----------------')
print(train_answers[0])

첫번째 샘플의 답변
-----------------
{'text': '교향곡', 'answer_start': 54}


현재 train_answers에는 정답이 본문 어디에 위치해 있는지 시작 인덱스는 있는데, 종료 인덱스는 없음. 종료 인덱스를 train_answers에 추가해줌

In [ ]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        answer['text'] = answer['text'].rstrip() # 뒤에 공백이 있다면 제거. ex)'1990년대 ' -> '1990년대'
        gold_text = answer['text'] # 정답.
        start_idx = answer['answer_start'] # 정답의 시작 인덱스
        end_idx = start_idx + len(gold_text) # 정답의 종료 인덱스를 계산

        assert context[start_idx:end_idx] == gold_text, "end_index 계산에 에러가 있습니다."
        answer['answer_end'] = end_idx

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [ ]:
print('첫번째 샘플의 답변')
print('-----------------')
print(train_answers[0])

첫번째 샘플의 답변
-----------------
{'text': '교향곡', 'answer_start': 54, 'answer_end': 57}


실제로 end_index가 맞는지 확인해봅시다.

In [ ]:
train_contexts[0][54:57]

'교향곡'

본문과 질문 두 가지를 함께 토크나이저의 입력으로 사용.

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('monologg/kobigbird-bert-base')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

Downloading:   0%|          | 0.00/236k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/480k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/169 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/373 [00:00<?, ?B/s]

[cls] 본문 토큰화 결과 [sep] 질문 토큰화 결과 [sep] pad

In [ ]:
print('첫번째 샘플의 토큰화 결과 :', train_encodings[0].tokens)

첫번째 샘플의 토큰화 결과 : ['[CLS]', '183', '##9', '##년', '바그너', '##는', '괴테', '##의', '파우', '##스트', '##을', '처음', '읽', '##고', '그', '내용', '##에', '마음', '##이', '끌려', '이를', '소재', '##로', '해서', '하나', '##의', '교향곡', '##을', '쓰', '##려', '##는', '뜻', '##을', '갖', '##는', '##다', '.', '이', '시기', '바그너', '##는', '183', '##8', '##년', '##에', '빛', '독촉', '##으로', '산전', '##수', '##전', '##을', '다', '[UNK]', '상황', '##이', '##라', '좌절', '##과', '실망', '##에', '가득', '##했', '##으며', '메', '##피스', '##토', '##펠', '##레스', '##를', '만나', '##는', '파우', '##스트', '##의', '심경', '##에', '공감', '##했', '##다고', '한다', '.', '또한', '파리', '##에서', '아브', '##네', '##크', '##의', '지휘', '##로', '파리', '음악', '##원', '관현악단', '##이', '연주', '##하', '##는', '베토벤', '##의', '교향곡', '9', '##번', '##을', '듣', '##고', '깊', '##은', '감명', '##을', '받', '##았', '##는데', ',', '이것', '##이', '이듬해', '1', '##월', '##에', '파우', '##스트', '##의', '서', '##곡', '##으로', '쓰여진', '이', '작품', '##에', '조금', '##이', '##라도', '영향', '##을', '끼쳤', '##으리', '##라는', '것', '##은', '의심', '##할', '여지', '##가', '없', '##다', '.', '여기', '##

In [ ]:
print('첫번째 샘플의 길이 :', len(train_encodings[0].tokens))

첫번째 샘플의 길이 : 4096


본문과 질문 사이에 [SEP]가 들어가있는 것을 확인할 수 있음.

In [ ]:
print('첫번째 샘플의 정수 인코딩', train_encodings[0].ids)

첫번째 샘플의 정수 인코딩 [2, 14295, 4550, 5161, 28014, 4741, 24918, 4561, 15352, 6927, 4604, 6962, 3733, 4625, 2656, 6988, 4584, 6914, 4544, 12326, 11662, 7841, 4618, 6876, 6838, 4561, 19358, 4604, 3554, 4626, 4741, 3033, 4604, 2554, 4741, 4742, 518, 3729, 7471, 28014, 4741, 14295, 4546, 5161, 4584, 3333, 27450, 9627, 21327, 4598, 4704, 4604, 2873, 1, 6895, 4544, 4744, 11982, 4676, 10714, 4584, 8250, 5411, 7022, 3178, 9145, 4590, 5410, 7568, 4874, 7224, 4741, 15352, 6927, 4561, 19807, 4584, 8841, 5411, 7939, 6787, 518, 6953, 8902, 31318, 29627, 4973, 4532, 4561, 8693, 4618, 8902, 7260, 4670, 32282, 4544, 8504, 4681, 4741, 18957, 4561, 19358, 529, 5371, 4604, 2962, 4625, 2676, 4839, 26597, 4604, 3246, 5091, 18150, 516, 7151, 4544, 11316, 521, 5170, 4584, 15352, 6927, 4561, 3417, 4854, 9627, 25301, 3729, 7113, 4584, 7239, 4544, 8350, 7136, 4604, 20358, 26196, 28147, 2579, 4839, 8550, 4647, 10612, 4771, 3614, 4742, 518, 6969, 4561, 3042, 4543, 4840, 7412, 4561, 6822, 9792, 2656, 4561, 7614, 4584, 1

In [ ]:
print('첫번째 샘플의 어텐션 마스크', train_encodings[0].attention_mask)

첫번째 샘플의 어텐션 마스크 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    deleting_list = []

    for i in tqdm(range(len(answers))):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        # 시작 인덱스가 비정상인 경우
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
            deleting_list.append(i)

        # 종료 인덱스가 비정상인 경우
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
            if i not in deleting_list:
              deleting_list.append(i)

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    return deleting_list

In [ ]:
deleting_list_for_train = add_token_positions(train_encodings, train_answers)
deleting_list_for_test = add_token_positions(val_encodings, val_answers)

100%|██████████| 5774/5774 [00:00<00:00, 206507.08it/s]


In [ ]:
print('삭제 예정인 훈련 샘플 :', deleting_list_for_train)
print('삭제 예정인 테스트 샘플 :', deleting_list_for_test)

삭제 예정인 훈련 샘플 : [13996]
삭제 예정인 테스트 샘플 : []


유일하게 인덱스가 비정상으로 추출된 샘플 데이터인 13996 데이터를 봄.

In [ ]:
print(train_contexts[13996])

이 영화 제작과는 별개로, 1990년대 중반에, 월트 디즈니 컴퍼니가 야마토의 영어 리메이크 버전인 스타 블레이저의 저작권을 사들였었는데 영화로 제작하지는 못했었다. TBS의 영화 제작을 담당하는 TBS 필름스는 2005년경부터 실사판을 계획하고 있었다. 2009년 7월 17일, 원래 '우주전함 야마토' 텔레비전 시리즈의 제작자이자 스태프인 이시구로 노보루‎는 그의 오타콘 패널에서 우주전함 야마토의 실사영화 버전을 개발 중임을 밝혔다. "꿈의 프로젝트"라 불리며 2010년 12월에 개봉 계획이고 22억 엔의 제작비인 이 프로젝트는, 《리터너》와 《올웨이즈 3번가의 석양》 시리즈로 잘 알려진 야마자키 다카시 감독이 맡기로 했다. 야마토는 개봉 전부터 어마어마한 제작비로 일본 영화계에 화두가 되었다.


In [ ]:
print('711번 샘플의 기존 정답 :', train_answers[13996])

711번 샘플의 기존 정답 : {'text': '이시구로 노보루\u200e', 'answer_start': 192, 'answer_end': 201}


In [ ]:
train_contexts[13996][192:210]

'이시구로 노보루\u200e는 그의 오타콘 '

In [ ]:
print('시작 인덱스 :', train_encodings['start_positions'][13996])
print('종료 인덱스 :', train_encodings['end_positions'][13996])

시작 인덱스 : 97
종료 인덱스 : 4096


노이즈가 있어 인덱스를 제대로 추출하지 못한 것으로 추정됨. 일단 무시하고 진행.

In [ ]:
def delete_samples(encodings, deleting_list):
  input_ids = np.delete(np.array(encodings['input_ids']), deleting_list, axis=0)
  attention_masks = np.delete(np.array(encodings['attention_mask']), deleting_list, axis=0)
  start_positions = np.delete(np.array(encodings['start_positions']), deleting_list, axis=0)
  end_positions = np.delete(np.array(encodings['end_positions']), deleting_list, axis=0)

  X_data = [input_ids, attention_masks]
  y_data = [start_positions, end_positions]

  return X_data, y_data

In [ ]:
X_train, y_train = delete_samples(train_encodings, deleting_list_for_train)
X_test, y_test = delete_samples(val_encodings, deleting_list_for_test)

In [ ]:
print('-------------삭제 전-------------')
print('훈련 데이터의 샘플의 개수 :', len(train_contexts))
print('테스트 데이터의 샘플의 개수 :', len(val_contexts))
print('-------------삭제 후-------------')
print('훈련 데이터의 샘플의 개수 :', len(X_train[0]))
print('테스트 데이터의 샘플의 개수 :', len(X_test[0]))

-------------삭제 전-------------
훈련 데이터의 샘플의 개수 : 60407
테스트 데이터의 샘플의 개수 : 5774
-------------삭제 후-------------
훈련 데이터의 샘플의 개수 : 60406
테스트 데이터의 샘플의 개수 : 5774


In [ ]:
X_train[0].shape

(60406, 4096)

In [ ]:
def get_text_data_by_idx(idx):
  print(tokenizer.decode(X_train[0][idx]))
  answer_encoded = X_train[0][idx][y_train[0][idx]:y_train[1][idx]+1]
  print(tokenizer.decode(answer_encoded))

In [ ]:
idx = 0
get_text_data_by_idx(idx)

[CLS] 1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 [UNK] 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡 ( 1악장 ) 을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다. [SEP] 바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

In [ ]:
idx = 7888
get_text_data_by_idx(idx)

[CLS] 20세기부터 기술의 발전에 따라 선선한 기후의 북쪽 지방 와인의 생산성이 높아지고, 품질 또한 개선되었다. 1938년부터 INAO ( Institut National des Appellations d'Origine, 국립원산지명칭위원회 ) 가 AOC ( Appellation d'origine controlee, 원산지 통제 명칭 ) 라는 규제를 만들었고, 와인양조 기준 ( 알코올 함량, 포도품종, 재배방법 ) 을 농산부령으로 관리하기 시작했다. AOC는 ‘ 샤블리 ’ 라는 이름을 보호하기 위해 만들어졌는데, 이미 그 당시에 ‘ 샤블리 ’ 라는 명칭이 전세계의 포도 품종에 부적적하게 사용되고 있었기 때문이다. 전세계적인 샤르도네 붐이 20세기 중반 일어남에 따라, 샤블리 포도밭은 다시 급격히 늘어나게 되었고, 2004년 약 4000 헥타르까지 확대되었다. [SEP] 샤블리 포도밭이 급격히 늘어나게 만든 붐은 무엇입니까? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [ ]:
# TPU 작동을 위한 코드 TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.108.243.226:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.108.243.226:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
from transformers import TFBertModel

In [ ]:
# 신규
class TFBertForQuestionAnswering(tf.keras.Model):
    def __init__(self, model_name):
        super(TFBertForQuestionAnswering, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.qa_outputs = tf.keras.layers.Dense(2,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                name='qa_outputs')
        self.softmax = tf.keras.layers.Activation(tf.keras.activations.softmax)

    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.bert(input_ids, attention_mask=attention_mask)
    
        sequence_output = outputs[0]

        logits = self.qa_outputs(sequence_output)
        start_logits, end_logits = tf.split(logits, 2, axis=-1)

        # start_logits = (batch_size, sequence_length,)
        # end_logits = (batch_size, sequence_length,)
        start_logits = tf.squeeze(start_logits, axis=-1)
        end_logits = tf.squeeze(end_logits, axis=-1)

        start_probs = self.softmax(start_logits)
        end_probs = self.softmax(end_logits)

        return start_probs, end_probs

In [ ]:
# 신규
with strategy.scope():
  model = TFBertForQuestionAnswering("monologg/kobigbird-bert-base")
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
  model.compile(optimizer=optimizer, loss=[loss, loss])

You are using a model of type big_bird to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'bert.pooler.weight', 'cls.predictions.bias', 'bert.embeddings.position_ids', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'bert.pooler.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initia

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=3,  # For demonstration, 3 epochs are recommended
    verbose=1,
    batch_size=8,
)

Epoch 1/3


7547/7547 [==============================] - 4981s 648ms/step - loss: 1.2882 - output_1_loss: 0.5868 - output_2_loss: 0.7014
Epoch 2/3
7547/7547 [==============================] - 4899s 649ms/step - loss: 0.7245 - output_1_loss: 0.3150 - output_2_loss: 0.4095
Epoch 3/3
4891/7547 [==================>...........] - ETA: 28:44 - loss: 0.5342 - output_1_loss: 0.2265 - output_2_loss: 0.3077

KeyboardInterrupt: ignored

In [ ]:
def predict_test_data_by_idx(idx):
  context = tokenizer.decode(X_test[0][idx]).split('[SEP] ')[0]
  question = tokenizer.decode(X_test[0][idx]).split('[SEP] ')[1]
  print('본문 :', context)
  print('질문 :', question)
  answer_encoded = X_test[0][idx][y_test[0][idx]:y_test[1][idx]+1]
  print('정답 :',tokenizer.decode(answer_encoded))
  output = model([tf.constant(X_test[0][idx])[None, :], tf.constant(X_test[1][idx])[None, :]])
  start = tf.math.argmax(tf.squeeze(output[0]))
  end = tf.math.argmax(tf.squeeze(output[1]))+1
  answer_encoded = X_test[0][idx][start:end]
  print('예측 :',tokenizer.decode(answer_encoded))
  print('----------------------------------------')

In [ ]:
for i in range(0, 100):
  predict_test_data_by_idx(i)

본문 : [CLS] 1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다. 
질문 : 임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은? 
정답 : 1989년 2월 15일
예측 : 1989년 2월 15일
----------------------------------------
본문 : [CLS] 1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학

In [ ]:
predict_test_data_by_idx(77)

본문 : [CLS] 기독교 성경 내용에는 모든 종들을 방주에 태운다고 이야기하고 있으나, 어류나 수중 생물에 대해서는 언급하지 않았다. 이것을 신학적 의미로만 받아들이면 괜찮은 문제이나, 이 현상이 실제로 일어났다고 가정할 경우, 이는 종 간 생존 환경의 차이에 대해서 간과하고 있다. 수중 생물이라 하더라도 종에 따라 생존할 수 있는 환경은 각각 다른 것이며, 40일 이내에 현존하는 가장 높은 산인 에베레스트 산도 잠기게 할 정도의 폭우로 인해 담수와 염수가 급작스럽게 섞일 경우, 급격한 삼투압 변화로 인해 대부분의 수생생물들이 폐사하게 되며, 결과적으로 육지 뿐 아니라 바다와 강의 모든 생태계가 파괴된다. 이후 5천년이라는 지극히 짧은 세월 동안 지구상의 동식물이 모두 페름기 대멸종 또는 K - T 대멸종에 준하는 대량절멸에 가까운 상태에서부터 시작하여 현재의 대략 870만 ( ±120만 ) 종에 달하는 생물다양성을 획득하려면 모든 생물들이 각 세대마다 종분화가 일어나야 할 만큼 엄청난 속도로 진화 및 번식이 ( 멸종 없이 ) 이루어져야만 가능한 일이다. ( 이와 관련하여 창조과학회 측에서는 북극곰의 예시를 통해 가지고 있던 특성이 없어지는 것이 진화가 아니라고 주장하지만, 통상적으로 알려진 바와 같이 생물학에서는 이미 존재하는 특성이 없어지는 현상, 즉 퇴화 역시 진화의 정의에 포함된다. ) 즉, 노아의 홍수가 실재하는 사건이었다면 진화적 종분화가 현재까지 알려진 것과 비교할 수 없이 엄청난 속도로 이루어져야만 현재 지구의 생물다양성을 설명할 수 있다. 게다가 이것은 현재의 생물종 멸종 속도를 전혀 고려하지 않았다. 다시 말해, 노아의 홍수가 실재하는 전지구적인 사건이기 위해서는 최소 캄브리아기 대폭발 수준의 폭발적인 진화적 종분화가 1 - 2억년이 아니라 최대 3 - 4천년 이내에 이루어졌어야만 현생 지구의 생물다양성에 대한 설명이 가능해진다. 그보다 더 중요한 것은, 각 동물들이 차지하는 영역과 먹이사슬에서의 위치, 375일 동안 먹이도 없이 밀폐된 공

In [ ]:
for i in range(600, 700):
  predict_test_data_by_idx(i)

본문 : [CLS] 서울시는 1969년 빈민주택의 불량함을 제거하고, 환경파괴나 불법점거 등의 주택단지 등을 없애기 위해 대대적으로 와우시민아파트 건립을 추진한다. 그러나 이 기공식에 있어서 완공까지의 기간이 얼마 안된것이 화근이었으니 1969 ~ 1970 공사기간 6개월이라는 공사기간으로 부실공사라는 화를 불러일으켰다. 완공 후 입주식 당일날 총 16개 동 중 1개동인 15동 및 14동에서 누수 및 벽, 기둥에 금이 가고 흔들리는 등 여러 문제가 발생하자 가장 불안한 15동 주민을 제외한 모든 주민들을 대피시키고 대피시킨 뒤 즉시 보수에 들어갔으나 그 과정에서 15동 주민들을 대피시키지 않았다. 결국 15동은 주저 않고 폭삭 무너졌으며, 74명의 주민들 중 30명 사망, 44명의 부상자를 냈다. 이는 건축과정의 시공감독 소홀, 심지어는 그것을 방관한 민, 관의 책임이라며 구청장, 감리, 건축과장 등의 인사들을 대거 개편하였다. 한달 뒤인 1970년 4월 당시 서울시장이었던 김현옥 시장이 결국 사건의 책임을 지고 자리를 물러나게 된다. 이후에도 여러 차례의 사고가 일어나 와우아파트는 시민아파트의 정리사업으로 철거되었다. 
질문 : 와우시민아파트 붕괴 사고 한달 뒤 자리에서 물러난 서울시장의 이름은? 
정답 : 김현옥
예측 : 김현옥
----------------------------------------
본문 : [CLS] 서울시는 1969년 빈민주택의 불량함을 제거하고, 환경파괴나 불법점거 등의 주택단지 등을 없애기 위해 대대적으로 와우시민아파트 건립을 추진한다. 그러나 이 기공식에 있어서 완공까지의 기간이 얼마 안된것이 화근이었으니 1969 ~ 1970 공사기간 6개월이라는 공사기간으로 부실공사라는 화를 불러일으켰다. 완공 후 입주식 당일날 총 16개 동 중 1개동인 15동 및 14동에서 누수 및 벽, 기둥에 금이 가고 흔들리는 등 여러 문제가 발생하자 가장 불안한 15동 주민을 제외한 모든 주민들을 대피시키고 대피시킨 뒤 즉시 보수에 들어갔으나 그 과정에